In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [3]:
len(df)

174643

In [4]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [ ]:
# examples with complex structure
print(data[19])
print(data[53365])

RT @AudrinaPatridge: @SofiaVergara is my favorite!!! #GoldenGlobes
OMG  RT @/moviefone: Adele, prepare to be the recipient of a very angry song from Taylor Swift #GoldenGlobes"


In [ ]:
import re

In [ ]:
retweets_freq_map = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @.*\b: ') 
   
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT ' prefix
        string = match.group()[3:]
        if string in retweets_freq_map:
            retweets_freq_map[string] += 1
        else:
            retweets_freq_map[string] = 1
        
    return re.sub(pattern, ' ', line)

# test
print(data[19])
print(remove_retweet_prefix(data[19]), '\n')
print(data[53365])
print(remove_retweet_prefix(data[53365]))

RT @AudrinaPatridge: @SofiaVergara is my favorite!!! #GoldenGlobes
 @SofiaVergara is my favorite!!! #GoldenGlobes 

OMG  RT @/moviefone: Adele, prepare to be the recipient of a very angry song from Taylor Swift #GoldenGlobes"
OMG   Adele, prepare to be the recipient of a very angry song from Taylor Swift #GoldenGlobes"


In [ ]:
def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line)

# test
print(data[19])
print(remove_retweet_prefix(data[19]))
print(cleanse(remove_retweet_prefix(data[19])), '\n')
print(data[53365])
print(remove_retweet_prefix(data[53365]))
print(cleanse(remove_retweet_prefix(data[53365])))

RT @AudrinaPatridge: @SofiaVergara is my favorite!!! #GoldenGlobes
 @SofiaVergara is my favorite!!! #GoldenGlobes
 @SofiaVergara is my favorite    #GoldenGlobes 

OMG  RT @/moviefone: Adele, prepare to be the recipient of a very angry song from Taylor Swift #GoldenGlobes"
OMG   Adele, prepare to be the recipient of a very angry song from Taylor Swift #GoldenGlobes"
OMG   Adele  prepare to be the recipient of a very angry song from Taylor Swift #GoldenGlobes 


In [ ]:
# test for #GoldenGlobes detection
count = 0
pattern = re.compile(r'\b#(.*)\b')
for tweet in data:
    match = re.search(pattern, tweet)
    if match:
        count += 1
        
        # check the first 10 cases
        if count < 10:
            print(match.group())

#GoldenGlobes red carpet thus far!http://t.co/NqcYL14r” #britsrule
#GoldenGlobes
#goldeglobes @goldenglobes @mcgregor_ewan :-) http://t.co/3WVXFjx1
#television#goodtime
#2013
#GOLDEN GLOBES
#GoldenGlobes but I hate Lucy Luis's dress!!! The emcees seem to love it!!! Who knows
#GoldenGlobes #GG2013
#GoldenGlobes


In [ ]:
# test for hashtag detection
count = 0
pattern = re.compile(r'\b@(.*)\b')
for tweet in data:
    match = re.search(pattern, tweet)
    if match:
        count += 1
        
        # check the first 10 cases
        if count < 10:
            print(match.group())

@SooyTuPadree TVES RT @Gheralber Televen RT@_YourVictim: Venevision. RT @VictorPulidoM ¿En que Canal dan los #GoldenGlobes
@FabSugar: Kerry Washington is EVERYTHING. Dying over her Miu Miu gown! #goldenglobes http://t.co/cYP5SYN7
@tomandlorenzo
@goldenglobes
@galleryId@/@galleryTitle@/@id@ via @eonline. I covet this outfit. I want
@JimmyChooLtd en los #goldenglobes 2013
@LeoDiCaprio at the #GoldenGlobes the old lady I work for says "Isn't he beautiful" #sheknowswhatsup
@NickFlora: Did Tina Fey come straight from a L'oreal commercial shoot? #GoldenGlobes
@DowntonAbbey  is already sweeping the #GoldenGlobes! I adore #MaggieSmith


In [ ]:
# potential extension:
# analysis solely dedicated to hashtags

In [ ]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [ ]:
def identify_entities(text):
    ents = list(nlp(text).ents)
    person_tags = {}
    tags = {}
    for ent in ents:    
        entity = ' '.join(t.orth_ for t in ent).strip()
        if entity not in tags:
            tags[entity]=[ent.label_]
            if ent.label_ == 'PERSON':
                person_tags[entity]=[ent.label_]
    return tags, person_tags

In [ ]:
# test
text = "“We’re going to keep things loose,” said Amy Poehler of her and co-host Tina Fey’s plan for the evening. #GoldenGlobes http://t.co/o40g5LGq“"
print('tags:', identify_entities(text))
print(identify_entities(text))

tags: ({'Amy Poehler': ['PERSON'], 'Tina Fey ’s': ['PERSON'], 'the evening': ['TIME'], 'GoldenGlobes': ['MONEY']}, {'Amy Poehler': ['PERSON'], 'Tina Fey ’s': ['PERSON']})
({'Amy Poehler': ['PERSON'], 'Tina Fey ’s': ['PERSON'], 'the evening': ['TIME'], 'GoldenGlobes': ['MONEY']}, {'Amy Poehler': ['PERSON'], 'Tina Fey ’s': ['PERSON']})


In [ ]:
def clean_entity(text):
    # remove_apostrophe
    if text.endswith("'s"):
        return text[:-2].strip()
    '''
    # remove tweet's hashtag prefix
    elif text.startswith('@'):
        return text[1:]
    # remove retweet's hashtag prefix
    elif text.startswith('RT @'):
        return text[4:]
    '''
    return text

# test
print(clean_entity("Tina 's"))
print(clean_entity("@goldenglobes"))
print(clean_entity('RT @accesshollywood'))

Tina
@goldenglobes
RT @accesshollywood


In [ ]:
def find_host(verbose=False):
    pattern = re.compile(r'host')
    entity_dict = {}
    
    num = 0
    for tweet in data:
        line = cleanse(remove_retweet_prefix(tweet))
        match = re.search(pattern, line.lower())
        if match:
            tags, person_tags = identify_entities(line)

            if verbose:
                # print the first 5 occurrences
                if num < 5:
                    print(line.strip())
                    print(tags)
                    print()

            for entity in tags.keys():
                entity = clean_entity(entity)
                if len(entity) > 1:
                    if entity not in entity_dict:
                        entity_dict[entity] = 1 # tried adding more weights to 'PERSON' tags but results are not good
                    else:
                        entity_dict[entity] += 1
            num += 1
            
    print('num of matches:', num)
    return entity_dict

In [ ]:
%%time
entity_dict = find_host(verbose=True)

Looking forward to watching Tina Fey and Amy Poehler host the #GoldenGlobes
{'Tina Fey': ['EVENT'], 'Amy Poehler': ['PERSON']}

It's our hosts Tina Fey and Amy Poehler  #goldenglobes #redcarpet http   t co 8lqC3ocQ
{'Tina Fey': ['PERSON'], 'Amy Poehler': ['PERSON'], '#': ['ORG']}

Tonight's dual hosting duties represent the culmination of a decade of Amy and Tina partnerships  http   t co scSThrn5 #GoldenGlobes
{'Tonight': ['TIME'], 'a decade': ['DATE'], 'Amy': ['PERSON'], 'Tina': ['GPE'], '': ['ORG']}

My green suede tuxedo pinching a bit here at the Velvet Rope Awards honoring best in crowd control  Topo Gigio   I hosting #GoldenGlobes
{'': ['NORP'], 'Topo Gigio': ['PERSON'], 'GoldenGlobes': ['MONEY']}

Tina Fey  amp  Amy Poehler Talk #GoldenGlobes Hosting  Drinking Game  http   t co 1mIvq362
{'Tina Fey': ['ORG'], 'Amy Poehler Talk': ['ORG'], 'GoldenGlobes Hosting': ['ORG'], 'Drinking Game': ['PERSON'], '': ['ORG']}



In [ ]:
# top retweets (each as an entity)
sorted(retweets_freq_map.items(), key=lambda pair: pair[1], reverse=True)[:50]

In [ ]:
top_100 = sorted(entity_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
top_100

In [ ]:
# pip3 install python-Levenshtein for 4-10x speedup
from fuzzywuzzy import fuzz

# test
print(fuzz.ratio('Tina Fey', 'Tina'))
print(fuzz.ratio('Amy Poehler', 'Amy'))
print(fuzz.ratio('Golden Globes', 'golden globes'))

In [ ]:
# remove golden globes from names
import pprint
names = [pair[0] for pair in top_100]

golden_globes = [name for name in names if fuzz.ratio(name.lower(), 'golden globes') > 50]
pprint.pprint(golden_globes)

In [ ]:
for name in golden_globes:
    del entity_dict[name]
top_20 = sorted(entity_dict.items(), key=lambda pair: pair[1], reverse=True)[:20]
top_20

In [ ]:
names = [pair[0] for pair in top_20]
names_clusters = []

for name in names:
    # each name starts as a cluster
    cluster = [name]
    names_to_reduce = names[:]
    names_to_reduce.remove(name)
    
    # one vs. all comparisons
    for i in names_to_reduce:
        ratio = fuzz.ratio(name.lower(), i.lower())
        # if similarity is larger than 50 or one name is contained in the other name
        if ratio > 50 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
            cluster.append(i)
    
    # if multiple names are identified in one cluster
    if len(cluster) > 1:
        names_clusters.append(cluster)
    
    print(cluster)

# find names clusters that should merge
# ['Amy Poehler', 'Amy', 'Amy Poelher']
# ['Tina', 'Tina Fey']
names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
names_clusters_reduced

In [ ]:
def evaluate_by_freq_and_weight(element):
    return entity_dict[element] * len(element)

e = entity_dict.copy()
for cluster in names_clusters_reduced:
    # select the longest name
    selected_name = max(cluster, key=evaluate_by_freq_and_weight)
    cluster.remove(selected_name)
    for name in cluster:
        # if not deleted in previous cases
        if name in e and selected_name in e:
            e[selected_name] += e[name]
            del e[name]

In [ ]:
top_10 = sorted(e.items(), key=lambda pair: pair[1])[-10:][::-1]
top_10

In [ ]:
# top 2 inferences for host
[name[0] for name in top_10][:2]